-----------
# Chapter 2 - Investment Decisions: The Certainty Case

-----------

In [2]:
using JuMP, HiGHS, Ipopt, Plots;

--------

#### 1
Basic capital budgeting problem with straight-line depreciation. The Roberts Company has
cash inflows of $140,000 per year on project A and cash outflows of $100,000 per year. The
investment outlay on the project is \\$100,000. Its life is 10 years. The tax rate, $τ_c$, is 40%. The
opportunity cost of capital is 12%.

(a) Present two alternative formulations of the net cash flows adjusted for the depreciation
tax shelter.

(b) Calculate the net present value for project A, using straight-line depreciation for tax
purposes.


In [52]:
T = 10
r = .12
Tc = .4

PV_A = (40000/r*(1-(1+r)^-T))*(1-Tc)+10000/r*(Tc)*(1-(1+r)^-T)-100000


58206.24479550426

-------

#### 2 
Basic capital budgeting problem with accelerated depreciation. Assume the same facts as in
Problem 1 except that the earnings before depreciation, interest, and taxes is $22,000 per year.

(a) Calculate the net present value, using straight-line depreciation for tax purposes.

(b) Calculate the net present value, using the sum-of-the-years digits method of accelerated
depreciation, for tax purposes.


In [19]:
PV_B = (22000/r*(1-(1+r)^-T))*(1-Tc)+10000/r*(Tc)*(1-(1+r)^-T)-100000

-2816.1639113330893

In [53]:
Dep_factor = zeros(T)
PV_Dep = zeros(T)
for i in 1:T
    Dep_factor[i] =  (10+1-i)/(sum(1:1:T))
    PV_Dep[i] = 100000*Dep_factor[i]*(1+r)^-i
end

PV_C = (22000/r*(1-(1+r)^-T))*(1-Tc)+PV_Dep'*Tc*ones(T)

100945.22865132123

--------
#### 3 
Basic replacement problem. The Virginia Company is considering replacing a riveting machine with a new design that will increase earnings before depreciation from $20,000 per year to
$51,000 per year. The new machine will cost $100,000 and has an estimated life of eight years,
with no salvage value. The applicable corporate tax rate is 40%, and the firm’s cost of capital is
12%. The old machine has been fully depreciated and has no salvage value. Should it be replaced
with a new machine?

In [58]:
CF_t = 51000-20000
CF_0 = -100000
T = 8
Tc = .4
r = .12

Depr_t = 100000/T

PV_mach = CF_0+CF_t/r*(1-(1+r)^-T)*(1-Tc)+Depr_t*Tc/r*(1-(1+r)^-T)

17236.29849739075

PV of new machine's cashflows is $17236.29. As this would add value to the company, we should replace the machine.

--------
#### 4 
Replacement problem when old machine has a positive book value. Assume the same facts as
in Problem 3 except that the new machine will have a salvage value of $12,000. Assume further
that the old machine has a book value of $40,000, with a remaining life of eight years. If replaced,
the old machine can, at present, be sold for $15,000. Should the machine replacement be made?


In [80]:
scrap = 12000

PV_new = CF_0+15000+(40000-15000)*Tc+CF_t/r*(1-(1+r)^-T)*(1-Tc)+((100000-scrap)/T-40000/T)*Tc/r*(1-(1+r)^-T)+scrap/(1+r)^T

34167.03383936283

---------
#### 5 
_Cash flows_. The Cary Company is considering a new investment that costs $10,000. It will
last five years and has no salvage value. The project would save $3,000 in salaries and wages each
year and would be financed with a loan with interest costs of 15% per year and amortization costs
(repayment of principal on the loan) of $2,000 per year. If the firm’s tax rate is 40% and its after-
tax cost of capital is 20%, what is the net present value of the project? (Note: The annuity factor
for five years at 20% is 2.991.)

In [91]:
CF_0 = -10000
CF_t = 3000
Tc = .4
r = .2
T = 5

PV_cary = CF_0+3000*1/r*(1-(1+r)^-T)*(1-Tc)-CF_0*Tc/(T*r)*(1-(1+r)^-T)

-2224.4084362139924

---------
#### 6 
Calculate the internal rate of return for the following set of cash flows:

$$
t1: 400 \\
t2 : 400 \\
t3: −1,000
$$

If the opportunity cost of capital is 10%, should the project be accepted?

In [8]:
model=Model(Ipopt.Optimizer)
@variable(model,i>=0)
@NLconstraint(model, 400/i*(1-(1+i)^-2)-1000*(1+i)^-3 == 0)
set_silent(model)
optimize!(model)

value.(i)

0.15831239269991165

IRR is 15.83%. As the opportunity cost of capital is 10%, we would reject this project as it does not represent efficient use of funds.

---------
#### 7 
Calculate the internal rate of return on the following set of cash flows:
$$
t0 : −1,000 \\
t1: 100 \\
t 2 : 900 \\
t 3: 100 \\
t 4 : −100 \\
t5 : −400
$$


In [16]:
model=Model(Ipopt.Optimizer)
@variable(model,i>=-10)
@NLconstraint(model, -1000 + 100*(1+i)^-1+900*(1+i)^-2+100*(1+i)^-3-100*(1+i)^-4-400*(1+i)^-5 == 0)
set_silent(model)
optimize!(model)

value.(i)

-1.9999999999999196

--------
#### 8 
The Ambergast Corporation is considering a project that has a three-year life and costs $1,200.
It would save $360 per year in operating costs and increase revenue by $200 per year. It would be
financed with a three-year loan with the following payment schedule (the annual rate of interestis 5%):
| Payment | Interest | Repayment of Principal | Balance |
| --- | --- | --- | --- | 
| 440.65 | 60.00 | 380.65 | 819.35 |
| 440.65 | 40.97 | 399.68 | 419.67 |
| 440.65 | 20.98 | 419.67 | 0 |
| | 121.95 | 1,200.00 | |

If the company has a 10% after-tax weighted average cost of capital, has a 40% tax rate, and uses
straight-line depreciation, what is the net present value of the project?

In [19]:
CF_0 = -1200
CF_t = 360+200
T = 3
r = .1
Tc = .4

Depr = -CF_0/T

PV = CF_0+CF_t/r*(1-(1+r)^-T)*(1-Tc)+Depr*Tc/r*(1-(1+r)^-T)

33.47858752817541

-------
#### 9 
The treasurer of United Southern Capital Co. has submitted a proposal to the board of
directors that, he argues, will increase profits for the all-equity company by a whopping 55%.
It costs $900 and saves $290 in labor costs, providing a 3.1-year payback even though the
equipment has an expected 5-year life (with no salvage value). If the firm has a 50% tax rate,
uses straight-line depreciation, and has a 10% weighted average cost of capital, should the project
be accepted? Income statements before and after the project are given in Tables Q9A and Q9B,
respectively.



<table>
<tr><th> **Table Q9A** </th><th> **Table Q9B** </th></tr>
<tr><td>

| Before | Year 1 | Year 2 | Year 3 | Year 4 | Year 5 | 
| --- | --- | --- | --- | --- | --- |                   
| Revenue | 1,000 | 1,000 | 1,000 | 1,000 | 1,000 |     
| Variable cost | 500 | 500 | 500 | 500 | 500 |         
| Depreciation | 300 | 300 | 300 | 300 | 300 |          
| Net operating income | 200 | 200 | 200 | 200 | 200 |  
| Interest expense | 0 | 0 | 0 | 0 | 0 |                
| Earnings before taxes | 200 | 200 | 200 | 200 | 200 | 
| Taxes | −100 | −100 | −100 | −100 | −100 |            
| Net income | 100 | 100 | 100 | 100 | 100 |            

</td><td>

| After | Year 1 | Year 2 | Year 3 | Year 4 | Year 5 |
| --- | --- | --- | --- | --- | --- |
| Revenue | 1,000 | 1,000 | 1,000 | 1,000 | 1,000 |
| Variable cost | 210 | 210 | 210 | 210 | 210 |
| Depreciation | 480 | 480 | 480 | 480 | 480 |
| Net operating income | 310 | 310 | 310 | 310 | 310 |
| Interest expense | 0 | 0 | 0 | 0 | 0 |
| Earnings before taxes | 310 | 310 | 310 | 310 | 310 |
| Taxes | −155 | −155 | −155 | −155 | −155 |
| Net income | 155 | 155 | 155 | 155 | 155 |

</td></tr> </table>



In [23]:
CF_0 = -900
CF_t = 290
r = .1
Tc = .5
T = 5

PV = CF_0 + CF_t/r*(1-(1+r)^-T)*(1-Tc)+900/(T*r)*Tc*(1-(1+r)^-T)

-9.16510918901406

Project would not provide positive value to the company & should therefore be rejected.

----------
#### 10 
The cash flows for projects A, B, and C are given below. 

| Year | A | B | C |
| --- | --- | --- | --- | 
| 0 | −1 | −1 | −1 |
| 1 | 0  | 1  | 0 |
| 2 | 2  | 0  | 0 |
| 3 | −1 | 1  | 3 |


Calculate the payback period and
net present value for each project (assume a 10% discount rate). If A and B are mutually exclusive
and C is independent, which project, or combination of projects, is preferred using 

(a) the payback method or
 
(b) the net present value method? What do the results tell you about the value-additivity properties of the payback method?

In [55]:
CF_A = [-1, 0, 2, -1]
CF_B = [-1, 1, 0, 1]
CF_C = [-1, 0, 0, 3]
CF_AC = CF_A+CF_C
CF_BC = CF_B+CF_C

PB_A = minimum(findall(x -> x .>= 0, cumsum(CF_A)))-1
PB_B = minimum(findall(x -> x .>= 0, cumsum(CF_B)))-1
PB_C = minimum(findall(x -> x .>= 0, cumsum(CF_C)))-1
PB_AC = minimum(findall(x -> x .>= 0, cumsum(CF_AC)))-1
PB_BC = minimum(findall(x -> x .>= 0, cumsum(CF_BC)))-1

[PB_A PB_B PB_C PB_AC PB_BC]

1×5 Matrix{Int64}:
 2  1  3  2  3

Project B is favoured in terms of Payback-Period.

In [54]:
r = .1

NPV_A = zeros(length(CF_A))
NPV_B = zeros(length(CF_B))
NPV_C = zeros(length(CF_C))
NPV_AC = zeros(length(CF_AC))
NPV_BC = zeros(length(CF_BC))

for i in 1:length(CF_A)
    NPV_A[i] = CF_A[i]*(1+r)^-(i-1)
    NPV_B[i] = CF_B[i]*(1+r)^-(i-1)
    NPV_C[i] = CF_C[i]*(1+r)^-(i-1)
    NPV_AC[i] = CF_AC[i]*(1+r)^-(i-1)
    NPV_BC[i] = CF_BC[i]*(1+r)^-(i-1)
end

ones(1,4)*[NPV_A NPV_B NPV_C NPV_AC NPV_BC]

1×5 Matrix{Float64}:
 -0.0984222  0.660406  1.25394  1.15552  1.91435

Project B has 2nd worst NPV, despite being the favoured project in terms of Payback-period.

The combination of projects B & C has the best NPV value depsite being the worst in terms of Payback-period. This is a result of the fact that Payback-period does not take into account the cashflows after the payback condition has been met.

Once Project C is combined with A or B, the results change if we use the payback criterion. Now
A + C is preferred. Previously, B was preferred. Because C is an independent choice, it should be
irrelevant when considering a choice between A and B. However, with payback, this is not true.
Payback violates value additivity. On the other hand, NPV does not. B + C is preferred. Its NPV is
simply the sum of the NPV’s of B and C separately. Therefore, NPV does obey the value additivity
principle.

#### 11
Calculate the internal rate of return on the following set of cash flows, according to the
economic interpretation of internal rate of return near the end of Section F.3. Assume that the
opportunity cost of capital is k = 10%.

| Year | Cash Flow |
| --- | --- |
| 0 | −5,000 |
| 1 | 10,000 |
| 2 | −3,000 |

In [64]:
model=Model(Ipopt.Optimizer)
@variable(model,i)
@NLconstraint(model, -5000+10000*(1+i)^-1-3000*(1+i)^-2 == 0)
set_silent(model)
optimize!(model)

value.(i)

0.6324555320336759

IRR is 63.25%, which is greater than the opportunity cost of capital of 10%. We thefore accept the project as it represents efficient use of funds.

Using the method discussed in section F.3 of this chapter, in the first year the firm invests $5,000 and
expects to earn IRR. Therefore, at the end of the first time period, we have 5,000(1 + IRR)

During the second period the firm borrows from the project at the opportunity cost of capital, k.
The amount borrowed is (10,000 − 5,000(1 + IRR))

By the end of the second time period this is worth:

(10,000 − 5,000(1 + IRR)) (1 + k)

The firm then lends 3,000 at the end of the second time period:

3,000 = (10,000 − 5,000(1 + IRR)) (1.10)
Solving for IRR, we have:

$\frac{\frac{3,000}{1.10}-10,000}{-5000}-1 = IRR = 45.45\% $